# Module 4 stuff

In [1]:
## Load relevant R packages.
install.packages("spdep") #***********UNCOMMENT THIS LINE IF RUNNING FOR FIRST TIME***********

inLibraries = list('repr','rgdal','spdep','classInt')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){           
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))      
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

library(readxl)
    library(tidyverse)
    library(data.table)
    library(dplyr)
    library(readr)
    library(ggplot2)
    install.packages("rworldmap")
    library(rworldmap)
    library(maptools)
    library(maps)
    library(RColorBrewer)
    library(repr)
    library(ggmap) #DataViz module requirement
    library(plotly) #DataViz module requirement
    library(ggthemes) #DataViz module requirement
    library(rgdal) #DataViz module requirement
    library(leaflet) #DataViz module requirement
    options(stringsAsFactors = FALSE)
    library(rgeos)
    library(spdep)
    library(classInt)

also installing the dependencies ‘gtools’, ‘gdata’, ‘spData’, ‘deldir’, ‘boot’, ‘LearnBayes’, ‘coda’, ‘expm’, ‘gmodels’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library(readxl)
library(tidyverse)
library(data.table)
library(dplyr)
library(readr)
library(ggplot2)
install.packages("rworldmap")
library(rworldmap)
library(maptools)
library(maps)
library(RColorBrewer)
library(repr)
library(ggmap) #DataViz module requirement
library(plotly) #DataViz module requirement
library(ggthemes) #DataViz module requirement
library(rgdal) #DataViz module requirement
library(leaflet) #DataViz module requirement
options(stringsAsFactors = FALSE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')
Checking rgeos availability: TRUE

Attaching package: ‘maps’

The following object is masked from ‘package:purrr’:

    map

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap")

In [3]:
 library(rgeos)

rgeos version: 0.4-3, (SVN revision 595)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.3-1 
 Polygon checking: TRUE 



In [4]:
loadFiles <- function(p) {
    if(FALSE) {
    "       
        Input: Directory of CSVs
        Output: Dataframe of merged CSVs
        Process: Loads all files from data folder into a list and merges all files into one data frame
        Verified by: BP, WCM, JAW, EWGS
        Author: EWGS
        Last Modified: 6/12
    "
    } #Documentation
    
    df <- list.files(path = p, pattern = ".csv", full.names = TRUE) %>% 
        lapply(read_csv, col_types = cols()) %>% #modified to supress the status messages 
        bind_rows
    
    message(paste("Files in \"", path, "\" loaded.", sep = ""))
    return(df)
}

reqCols <- function(df) {
    if(FALSE) {
    "
        Input: Raw dataframe
        Output: Subset of dataframe for needed columns
        Process: Hardcoded columns to be used, na omitted, deduplicated rows
        Verified by: BP, WCM, JAW, EWGS
        Author: JAW
        Last Modified: 6/12
    "
    } #Documentation

    df <- subset(df, select = c(
        'time',
        'latitude',
        'longitude',
        'depth',
        'mag',
        'id',
        'updated'
    ))
    df <- na.omit(df)
    df <- unique(df)
    
    message("Subset Completed.")
    return(df)
}

parseDt <- function(df, col) {
    if(FALSE) {
    "
        Input: Dataframe with a POSIXct datetime column
        Output: Adds a column with the date, time, year into their own columns
        Process: Isolates the dates and times from df$time and df$updated into their own columns.
        Verified by: BP, WCM, JAW, EWGS
        Author: EWGS
        Last Modified: 6/12
    "
    } #Documentation
    
    #New columns will be named by: col + "_suffix"
    newDate = paste(col, "Date", sep = "_")
    newTime = paste(col, "Time", sep = "_")
    newYear = paste(col, "Year", sep = "_")
    
    df[, newDate] <- as.Date(df[[col]])
    df[, newTime] <- as.ITime(strftime(df[[col]], format = "%H:%M:%S"))
    df[, newYear] <- as.numeric(format(df[[col]], '%Y'))
    
    message(paste("Parsed column: ", col, ".", sep = ""))
    return(df)
}

tospdf <- function(df) {
    if(FALSE) {
    "
        Input: Raw dataframe
        Output: Subset of dataframe for needed columns
        Process: Hardcoded columns to be used, na omitted, deduplicated rows
        Verified by: 
        Author: JAW
        Last Modified: 6/13
    "
    } #Documentation
    df <- na.omit(df)
    df <- subset(df, select = c(
        'time',
        'latitude',
        'longitude',
        'depth',
        'mag',
        'id',
        'updated'
    ))
#we select all the columns from the general df

lats <- df$latitude
longs <- df$longitude
#we want to pick out the lat lon cordinates to store the location of each of the df attributes

quakeattributes <- df[,c('depth','mag','time', 'id', 'updated')]
#here we then assign the rest of the coulmns to the attributes
    
adddf <- as.data.frame(quakeattributes)
#make attributes into it's own df
    
ptcoords <- cbind(as.numeric(longs),as.numeric(lats))
#we want to perpare the cordinates for our SpatialPointsDataFrame function
    
spts <- SpatialPointsDataFrame(ptcoords,data=adddf, proj4string=CRS("+proj=longlat +ellps=WGS84"))
#we now have a spatial points df that we can use to create maps and in our analysis
     message(paste("spatial df created"))
    return(spts)
}


In [5]:
path <- "~/jupyter/cs2019_Group11/GroupProducts/data" #Explicit location of data

df <- loadFiles(path) %>%
    reqCols %>%
    parseDt('time')

dfsp <- tospdf(df)

Files in "~/jupyter/cs2019_Group11/GroupProducts/data" loaded.
Subset Completed.
Parsed column: time.
spatial df created


In [6]:
#jessica

tpts <- spTransform(dfsp,  CRS("+init=epsg:4087"))
#transforming the data for stats

In [7]:
#jessica

tectonicdata = "fe.kmz"
tectonicFeatures <- readOGR(tectonicdata)
#loading in the data for Flinn-Engdahl Regions, to be used later

OGR data source with driver: LIBKML 
Source: "/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

In [8]:
#Jessica

transTectonicFeatures <- spTransform(tectonicFeatures,  CRS("+init=epsg:4087"))
#transform the Flinn-Engdahl Regions to match the spatial dataframe

In [9]:
#jessica
library(rgeos)
#add empty names row to be added to 
tpts@data$newname = "empty"

#start for loop to go through each seismic homogeneous region
for (i in 1:nrow(transTectonicFeatures)) {
    region = transTectonicFeatures[i, ]  

#here we determine which region each earthquake belongs to
    selFeat <- overGeomGeom(region, tpts, returnList = TRUE, fn = NULL)

#Unlist the list
    intSet = unlist(selFeat)
    
#print to check to ensure that it ran correctly
    print(i)
    
#we want to now add an attribute that captures which region each earthquake belongs to
    #to be named there must be at least one earthquake 
    if (length(intSet) > 0){
        tpts@data[intSet,]$newname <- i}
    #if no earthquakes in the region then none are named
    else {
        print("none")}
     }

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] "none"
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] "none"
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] "none"
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] "none"
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] "none"
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] "none"
[1

In [11]:
tpts_df <- as.data.frame(tpts)
tpts_df$latitude = df$latitude
tpts_df$longitude = df$longitude
head(tpts_df)

depth,mag,time,id,updated,newname,coords.x1,coords.x2,latitude,longitude
2.000,2.6,1999-07-01 23:29:27,usp0009awp,2014-11-07 01:08:01,374,421900.9,5059471,45.4500,3.790
4.613,3.1,1999-07-01 22:10:51,uw10474128,2016-07-23 18:59:57,6,-13667584.4,5199789,46.7105,-122.778
106.800,4.2,1999-07-01 21:42:41,usp0009awk,2014-11-07 01:08:01,1,19811752.4,5788057,51.9950,177.972
26.100,3.9,1999-07-01 20:50:11,usp0009awh,2014-11-07 01:08:01,53,-8020903.3,-3531722,-31.7260,-72.053
136.300,3.1,1999-07-01 19:52:19,usp0009awf,2014-11-07 01:08:01,24,-17094221.0,6666924,59.8900,-153.560
30.200,3.8,1999-07-01 19:52:12,usp0009awe,2014-11-07 01:08:01,741,3469383.3,4013068,36.0500,31.166


In [12]:
#grouping and summarising by max mag
tpts_df_mm = tpts_df %>% group_by(newname) %>% summarise(max(mag))

#changing column name to max_mag for easier use
colnames(tpts_df_mm) <- c('newname', 'max_mag')

#ordering by descending max_mag
tpts_df_mm = arrange(tpts_df_mm, desc(max_mag))

#displaying top then regions and their max_mags
top_ten_mag_regions = head(tpts_df_mm, 10)
top_ten_mag_regions

newname,max_mag
197,9.1
570,9.1
54,8.8
593,8.6
191,8.4
65,8.4
171,8.3
553,8.3
669,8.3
60,8.2


In [13]:
tpts_df_mm = tpts_df %>% group_by(newname) %>% summarise(max(mag))

#changing column name to max_mag for easier use
colnames(tpts_df_mm) <- c('newname', 'max_mag')

#ordering by descending max_mag
tpts_df_mm = arrange(tpts_df_mm, max_mag)

#displaying bottom then regions and their max_mags
bottom_ten_mag_regions = head(tpts_df_mm, 10)
bottom_ten_mag_regions

newname,max_mag
518,2.60
597,2.60
430,2.70
478,2.80
400,2.90
622,2.90
425,2.95
740,2.98
384,3.00
387,3.00


In [ ]:
#I am now going to calculate the nearest neighbor stats for each of the Flinn-Engdahl Regions that contian 
#the highest magitude earthquakes, I am going to do those because juypter does not have enough meomry to run
#all of the regions

In [41]:
#jessica

n = 197
#n is the region which I am wanting to look at

    region197 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion197 <- knn2nb(knearneigh(region197, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist197 <- max(unlist(nbdists(nieghRegion197, region197)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb197 <- dnearneigh(region197, 0, unlist197)
#now i am going to determine the neighbours of region points by Euclidean distance


In [42]:
#jessica

summary(gc.nb197)
#print to get general idea about the data


Neighbour list object:
Number of regions: 5314 
Number of nonzero links: 391768 
Percentage nonzero weights: 1.387349 
Average number of links: 73.72375 
Link number distribution:

  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
  4  14  15  25  30  33  33  27  32  26  28  41  36  40  30  36  23  24  29  17 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
 28  31  27  43  25  36  34  49  37  35  47  31  30  38  47  35  34  48  35  46 
 41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60 
 25  41  38  33  34  35  36  30  40  27  38  37  29  28  46  35  47  48  47  51 
 61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80 
 51  45  61  54  50  44  51  44  47  61  60  49  63  54  47  53  61  59  54  57 
 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 
 62  59  59  55  56  49  52  42  49  54  46  56  52  51  42  44  46  46  49  37 
101 102 1

In [43]:
#jessica

n = 570
#n is the region which I am wanting to look at

    region570 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion570 <- knn2nb(knearneigh(region570, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist570 <- max(unlist(nbdists(nieghRegion570, region570)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb570 <- dnearneigh(region570, 0, unlist570)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb570)
#print to get general idea about the data


Neighbour list object:
Number of regions: 2318 
Number of nonzero links: 400456 
Percentage nonzero weights: 7.452945 
Average number of links: 172.7593 
Link number distribution:

  1   2   3   4   5   8  10  11  12  13  14  15  16  17  18  19  20  21  22  23 
  2   2   1   8   2   2   2   3   1   3   4   2   7   4   3   3   3   2   3   3 
 24  25  26  27  28  29  30  32  33  34  35  36  37  38  39  40  41  42  43  44 
  7   2   2   3   2   5   3   2   2   3   2   5   8   3   5   5   4   6   2   4 
 45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64 
  2   5   2   7   9  13  22  11   8  10  12   9   4   3   8   6   2   2   6   9 
 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84 
  5   7   8   9   5   4   3   6   5   5   9  12   9  16  10   8  20  11  11  14 
 85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 
 17  13  10   7  11  16  13  13  17  19  21   8  13  15  20  10  25  11  16  14 
105 106 1

In [44]:

#jessica

n = 54
#n is the region which I am wanting to look at

    region54 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion54 <- knn2nb(knearneigh(region54, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist54 <- max(unlist(nbdists(nieghRegion54, region54)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb54 <- dnearneigh(region54, 0, unlist54)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb54)
#print to get general idea about the data


Warning message in knearneigh(region54, k = 1):
“knearneigh: identical points found”

Neighbour list object:
Number of regions: 10273 
Number of nonzero links: 7122784 
Percentage nonzero weights: 6.749245 
Average number of links: 693.3499 
Link number distribution:

   1    2    3    4    5    6    8    9   10   11   13   14   15   16   17   18 
   3    5    4    4    1    1    1    1    1    1    1    1    1    3    2    1 
  19   20   21   22   23   24   25   26   27   28   29   31   32   33   34   35 
   3    2    3    4    2    1    1    1    2    1    1    5    4    5    9    1 
  36   37   38   40   41   42   43   44   45   46   47   48   49   50   51   52 
   2    2    4    4    3    2    6    5    4    2    3    2    7    6   10    2 
  53   54   55   56   57   58   59   60   62   63   64   65   66   67   68   69 
  10    5    5    7    4    4    5    7   11   10    6    4    8   10    5    4 
  70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85 
  10    6    3   10    7    6    7    9    7    5   14   12    9   14   10   14 
  86   

In [45]:
#jessica

n = 593
#n is the region which I am wanting to look at

    region593 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion593 <- knn2nb(knearneigh(region593, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist593 <- max(unlist(nbdists(nieghRegion593, region593)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb593 <- dnearneigh(region593, 0, unlist593)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb593)
#print to get general idea about the data


Warning message in knearneigh(region593, k = 1):
“knearneigh: identical points found”

Neighbour list object:
Number of regions: 4102 
Number of nonzero links: 4216618 
Percentage nonzero weights: 25.05953 
Average number of links: 1027.942 
Link number distribution:

   1    3    4    7   10   11   14   20   22   39   48   52   53   58   66   69 
   1    1    1    1    1    2    1    1    1    1    1    1    1    1    1    1 
  73   79   80   84   87   88   89   90   91   92   93   94   98   99  100  103 
   1    1    2    2    1    2    2    2    4    1    2    1    1    2    2    5 
 104  105  106  108  110  111  112  113  117  119  120  122  123  124  125  126 
   1    1    2    3    4    1    1    1    2    1    4    3    2    2    2    2 
 127  128  129  130  131  132  133  134  135  136  139  140  142  144  147  148 
   3    4    4    5    1    1    2    1    2    3    1    6    1    1    2    1 
 152  154  157  160  161  162  163  164  165  166  167  168  169  171  172  174 
   1    1    2    1    3    1    2    1    1    1    1    1    1    3    2    1 
 175  17

In [46]:
#jessica

n = 65
#n is the region which I am wanting to look at

    region65 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion65 <- knn2nb(knearneigh(region65, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist65 <- max(unlist(nbdists(nieghRegion65, region65)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb65 <- dnearneigh(region65, 0, unlist65)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb65)
#print to get general idea about the data


Neighbour list object:
Number of regions: 1475 
Number of nonzero links: 93252 
Percentage nonzero weights: 4.286217 
Average number of links: 63.22169 
Link number distribution:

  1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21 
  4   4   3   3   5  11   4  10   7   8  16  17   8   7  16  14  24   7   7  15 
 22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41 
 17  13  15  12  14  29  20  19  20  16  16  17  12  19  17  16  14  12  21  14 
 42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61 
 10  12  14   4  11  11  17   9  10   9  12  16  11  14  11  12   6  13  12  18 
 62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81 
 11  13  20  16  17  10  14  10  13  15  13  11  12  18  15  19  16  14  11  12 
 82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 
  7   9  13  16   5   4   7   8   5  14   7  13   1   8  10   5   5   8   7   6 
102 103 10

In [47]:
#jessica

n = 171
#n is the region which I am wanting to look at

    region171 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion171 <- knn2nb(knearneigh(region171, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist171 <- max(unlist(nbdists(nieghRegion171, region171)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb171 <- dnearneigh(region171, 0, unlist171)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb171)
#print to get general idea about the data


Warning message in knearneigh(region171, k = 1):
“knearneigh: identical points found”

Neighbour list object:
Number of regions: 4443 
Number of nonzero links: 732614 
Percentage nonzero weights: 3.71127 
Average number of links: 164.8917 
Link number distribution:

  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
  3   3   2   2   5   8   5   6   7   1   5   8  12   5   8   7   9   8  12  16 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
 12   8  15  16  15  14  17  19  20  27  20  23  28  12  25  21  21  20  29  28 
 41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60 
 28  23  23  27  34  25  27  20  20  21  15  29  20  25  28  21  18  26  14  18 
 61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80 
 11  25  19  31  21  24  24  16  26  21  23  26  21  21  12  22  25  10  13  18 
 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 
 16  12  14  16  13  15  13  11  11  14   8  18  12  18  23  14  14   9  16  16 
101 102 10

In [48]:
#jessica

n = 553
#n is the region which I am wanting to look at

    region553 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion553 <- knn2nb(knearneigh(region553, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist553 <- max(unlist(nbdists(nieghRegion553, region553)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb553 <- dnearneigh(region553, 0, unlist553)
#now i am going to determine the neighbours of region points by Euclidean distance

    
summary(gc.nb553)
#print to get general idea about the data


Neighbour list object:
Number of regions: 237 
Number of nonzero links: 30808 
Percentage nonzero weights: 54.84876 
Average number of links: 129.9916 
Link number distribution:

  1   2  71  74  76  93  99 101 103 104 105 106 107 124 126 127 128 129 130 131 
  1   3   1   1   2   2   1   3   6   3  36   3  14   2  10   2  15   4   3   2 
132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 148 149 150 151 152 
  2  16   5   2   4   1   7   2   1   1   2   5   1   3  10   2   2   1   2   2 
154 155 156 159 160 161 162 164 168 172 173 175 176 192 193 194 196 201 203 
  3   5   2   2  21   2   2   1   2   2   1   1   1   1   1   2   1   1   1 
1 least connected region:
61 with 1 link
1 most connected region:
159 with 203 links


In [49]:
#jessica

n = 60
#n is the region which I am wanting to look at

    region60 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion60 <- knn2nb(knearneigh(region60, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist60 <- max(unlist(nbdists(nieghRegion60, region60)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb60 <- dnearneigh(region60, 0, unlist60)
    #now i am going to determine the neighbours of region points by Euclidean distance

summary(gc.nb60)
#print to get general idea about the data


Neighbour list object:
Number of regions: 1573 
Number of nonzero links: 36868 
Percentage nonzero weights: 1.49002 
Average number of links: 23.43802 
Link number distribution:

 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 
 3  5  4 14 14 25 29 28 50 41 45 49 67 77 60 62 53 60 49 46 48 58 48 48 45 36 
27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 
30 35 36 30 29 23 26 25 26 19 13 11 11  8 15 13 13  8  8 12  5 11  3  9  3  7 
53 54 55 56 57 58 59 60 61 62 63 64 65 67 68 69 
 5  7  2  7  7  9 11  4  8  2  3  4  5  3  2  1 
3 least connected regions:
181 253 1264 with 1 link
1 most connected region:
697 with 69 links


In [ ]:
#now i have some spatial stats on the earthquakes and a general idea abput how connected/related the data is 

In [ ]:
#I am now going to calculate the nearest neighbor stats for each of the Flinn-Engdahl Regions that contian 
#the lowest magitude earthquakes, I am going to do those because juypter does not have enough meomry to run
#all of the regions

In [99]:
n = 597
#n is the region which I am wanting to look at

    region60 <- subset(tpts, newname == n)
#I only want earthquakes in that region 

    nieghRegion60 <- knn2nb(knearneigh(region60, k=1))
#I am now going to determine the maximum distance to the closest point

    unlist60 <- max(unlist(nbdists(nieghRegion60, region60)))
#I am going to now use that distance as the thershold so that every point has at least on neighbor

    gc.nb60 <- dnearneigh(region60, 0, unlist60)
    #now i am going to determine the neighbours of region points by Euclidean distance

summary(gc.nb60)
#print to get general idea about the data


Warning message in knearneigh(region60, k = 1):
“k greater than one-third of the number of data points”

Neighbour list object:
Number of regions: 2 
Number of nonzero links: 2 
Percentage nonzero weights: 50 
Average number of links: 1 
Link number distribution:

1 
2 
2 least connected regions:
1 2 with 1 link
2 most connected regions:
1 2 with 1 link


ERROR: Error in bind_rows(listofdfs): could not find function "bind_rows"
